In [2]:
import tensorflow as tf 
import os
from tensorflow.keras import layers
# from keras import layers

In [4]:
os.getcwd()

'c:\\Users\\arham\\Acute Lymphoblastic Leukemia\\research'

In [5]:
dataDir = '../artifacts'
data = tf.keras.preprocessing.image_dataset_from_directory(
    dataDir,
    shuffle=True,
    image_size=(224,224),
    batch_size=16
)

Found 3256 files belonging to 4 classes.


In [7]:
len(data)

204

In [8]:
total_batch = len(data)

train_size = int(total_batch*0.7)+1
val_size = int(total_batch*0.2)+1
test_size = int(total_batch*0.1)

In [9]:
train_size, test_size, val_size

(143, 20, 41)

In [10]:
total_batch

204

In [11]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Resizing, Rescaling

In [13]:
resize_and_rescale = tf.keras.Sequential([
  Resizing(256, 256),
  Rescaling(1./255),
])

In [14]:
model = tf.keras.Sequential([
    # resize_and_rescale,
    resize_and_rescale,
    layers.Conv2D(32, (3,3), 1, activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), 1, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), 1, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), 1, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, (3,3), 1, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),

    layers.Dense(512, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(18, activation='relu'),
    layers.Dense(9, activation='softmax')
])

# Compiling the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.build(input_shape=(None, 224, 224, 3))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)       

In [ ]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [32]:
vgg = VGG16(include_top=False, weights='imagenet')

# Freeze the layers
for layer in vgg.layers:
    layer.trainable = False

# Print a summary of the model to confirm that the layers are frozen
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [40]:
resize_and_rescale = tf.keras.Sequential([
  Resizing(256, 256),
  Rescaling(1./255),
])

In [57]:
os.path.dirname('../artifacts/model.h5')

'../artifacts'

In [58]:
def build_model(): 
     
    input_layer = Input(shape=(224,224,3))
    
    vgg = VGG16(include_top=False, weights='imagenet')
    for layer in vgg.layers:
        layer.trainable=False
    
    vgg = vgg(input_layer)
    f1 = GlobalMaxPooling2D()(vgg)
    layer1 = Dense(256, activation='relu')(f1)
    layer2 = Dense(128, activation='sigmoid')(layer1)
    layer3 = Dense(64, activation='sigmoid')(layer2)
    layer4 = Dense(16, activation='sigmoid')(layer3)
    layer5 = Dense(8, activation='sigmoid')(layer4)
    output = Dense(4, activation='softmax')(layer5)
    
    
    
    model = Model(input=input_layer, outputs=output)
    return model

In [55]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_max_pooling2d_7 (Gl  (None, 512)               0         
 obalMaxPooling2D)                                               
                                                                 
 dense_36 (Dense)            (None, 256)               131328    
                                                                 
 dense_37 (Dense)            (None, 128)               32896     
                                                                 
 dense_38 (Dense)            (None, 64)                8256      
                                                           

In [38]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [39]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
143/143 [==============================] - 606s 4s/step - loss: 1.3355 - accuracy: 0.3291 - val_loss: 1.1329 - val_accuracy: 0.4436
Epoch 2/20
 54/143 [==========>...................] - ETA: 5:15 - loss: 1.0599 - accuracy: 0.4873

KeyboardInterrupt: 

In [45]:
model = tf.keras.Sequential([
    # resize_and_rescale,
    resize_and_rescale,
    vgg])


In [48]:
model.build(input_shape=(None, 224, 224, 3))

In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
